**잠재디리클레할당(Latent Dirichlet Allocation, LDA) a.k.a Topic modling**

: LDA란 주어진 문서에 대하여 각 문서에 어떤 주제들이 존재하는지에 대한 확률모형, 즉 특정 토픽에 특정 단어가 나타날 확률을 나타냄

![image](https://user-images.githubusercontent.com/29038531/111445738-0ae6dc00-874f-11eb-9aea-4b528925e5ef.png)

쉽게 말해 문서가 있고, 그 안에 단어가 있다면 확률분포 중 하나인 디리클레의 분포를 가정하고, 번호가 매겨진 토픽 안에 문서와 단어들을 하나씩 넣어보며 잠재적인 의미(토픽)들을 찾아주는 과정

**1. LDA 이론**

1번 문서 : 문고리거래 하실분
2번 문서 : 가방 나눔합니다. 문고리드림
3번 문서 : 비대면거래로 합니다. 택배로 할께요.

1번 문서 : 문고리, 거래
2번 문서 : 가방, 나눔, 문고리, 드림
3번 문서 : 비대면, 거래, 택배

<img width="909" alt="스크린샷 2021-03-17 오후 6 43 45" src="https://user-images.githubusercontent.com/29038531/111447367-b6dcf700-8750-11eb-85e5-0434fecf8d61.png">

<img width="900" alt="스크린샷 2021-03-17 오후 6 45 19" src="https://user-images.githubusercontent.com/29038531/111447597-ef7cd080-8750-11eb-89cc-1d3876383032.png">

1번문서의 문고리 단어가 topic1로 배정되어 1의 값이 생기고, LDA 파라미터 값인 α를 0.01로 설정하면 그 값을 같이 더해줍니다. 그렇게 1번문서에는 topic3에 배정된게 없지만 최소한의 값(α)을 배정해줍니다.

<img width="896" alt="스크린샷 2021-03-17 오후 6 47 29" src="https://user-images.githubusercontent.com/29038531/111447904-3bc81080-8751-11eb-9b64-dab64918beef.png">

<img width="904" alt="스크린샷 2021-03-17 오후 6 48 11" src="https://user-images.githubusercontent.com/29038531/111447981-55695800-8751-11eb-8448-c856813fff98.png">

해당 과정에서부터는 1번 문서에 속한 ‘문고리’의 키워드를 topic1~topic3에 확률을 계산합니다.

1번 문서 내 topic1이 있을 확률 : 1.01/3.03 = 0.333
(이 때 분모의 경우 1번 토픽 내 문서 분포 합입니다. )

1번 토픽 내 단어가 ‘문고리’일 확률 : 1.001/3.007 = 0.332
(이 때 분모의 경우 1번 토픽 내 키워드 분포 합입니다.)
마지막으로 1번 문서의 ‘문고리’가 topic1일 확률 0.333*0.332 = 0.110이 됩니다.

위처럼 1번 문서 내 topic2이 있을 확률,
1번 문서 내 topic3이 있을 확률 모두 구해줍니다.

(1번 분서 내 topic2 : 1.01 / 3.03 = 0.333, 2번 토픽 내 단어 "문고리" : 1.001 / 4.008 = 0.249, 1번 문서의 ‘문고리’가 topic2일 확률 = 0.082)

 (1번 분서 내 topic3 : 0.01 / 3.03 = 0.030, 3번 토픽 내 단어 "문고리" : 0.001 / 2.007 = 0.0004, 1번 문서의 ‘문고리’가 topic2일 확률 = 0.000012)


In [50]:
# ================================================== [ setting ] ==================================================
!pip install konlpy
!pip install pyLDAvis
import pandas as pd 
import re
from konlpy.tag import Komoran
from gensim import corpora
import gensim
import pyLDAvis.gensim
import warnings
warnings.filterwarnings(action = 'ignore')

news_data = pd.read_csv("kyungnam_news_total.csv", encoding = "utf-8")

In [35]:
# news_data["new_summary"][3] 
news_data["new_summary"].str.replace("\([^)]*\)", "")[3]

'창원시 조영진 제1부시장은 24일 창원컨벤션센터에서 열리고 있는 ‘제13회 2020 한국국제기계박람회’ 행사장을 찾았다.이번 행사는 기계와 DNA 생태계 융합을 통해 기계산업 분야의 새로운 청사진을 제시하고 있다.한국기계산업진흥회가 주관하고 경상남도와 창원시가 주최하는 KI'

In [39]:
# news_data["new_summary"][411]
news_data["new_summary"].str.replace("\n\n", "")[411]
news_data["new_summary"].str.replace("[^a-zA-Z가-힣 ]", "")[411]

'링을 선사한다 지난 년대 마산의 부흥과 함께 급격한 도시화가 이뤄진 구암동은 도시재생 뉴딜사업으로 또 한 번의 변화를 겪고 있다 구암지구에는 년까지 총사업비 억 원이 투입돼 '

In [41]:
news_data["new_summary"][120]
# news_data["new_summary"].str.replace("\([^)]*\)", "")[120] # (주) : 특문
news_data["new_summary"].str.replace("[^a-zA-Z가-힣 ]", "")[120]

'경남도는 일 도청에서 NHN HDC현대산업개발 김해시와 함께 NHN 클라우드 데이터센터 건립을 위한 투자협약 체결식을 개최했다투자협약 체결식에는 김경수 지사를 비롯해 박영선 중소벤처기업부장관 진은숙 NHN CTO총괄이사 권순호 HDC현대산업개발 대표이사 허성곤 김해시장 등 관계자들이 참석했다이번 협약MOU의 체결로 NHN과 HDC현대산업개발은 김해시 부원지구 일원 약 만을 공동으로 개발하고 약 만의 부지면적에 억 원을 투입한다오는 년까지 클라우드 기반 도심형 데이터'

In [51]:
# ============================================== [ LDA modeling ] ======================================================
def LDA_modeling(df) :
  df = df.str.replace("\([^)]*\)", "") # 괄호안의 내용 제거 ex) 코로나(COVID-19) -> 코로나
  df = df.str.replace("\n\n", "") # 문단 구분자 제거 
  df = df.str.replace("[^a-zA-Z가-힣 ]", "") # 영어 / 한글 제외하고 모두 제거 

  processed_data = []
  for sentence in df : 
    komoran = Komoran()
    tag = komoran.pos(sentence) # 형태소 처리 ["단어", "품사"]
    nouns = [s for s, t in tag if t in ['SL', 'NNG', 'NNP'] and len(s) > 1] # 명사 태그 추출, 한글자 이상만 추출 # nng : 명사 nnp :  고유대명사 sl : 외래어  
    processed_data.append(nouns) # token 형태로 list에 저장

  dictionary = corpora.Dictionary(processed_data) # {512, 1} {522, 2} ... 
  corpus = [dictionary.doc2bow(text) for text in processed_data]
  ## ===== 요기 까지 전처리 ==============================================

  NUM_TOPICS = 3
  ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word = dictionary, passes = 15)
  topics = ldamodel.print_topics(num_words = 10)
  
  pyLDAvis.enable_notebook()
  vis = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary) # 그래프 출력
  
  return pyLDAvis.display(vis)

In [ ]:
LDA_modeling(news_data["new_summary"])